<a href="https://colab.research.google.com/github/DesmondWasHere/colab-scraping/blob/main/TOI_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Downloading Selenium and chrome webdriver.
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 7.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launch

In [ ]:
## Header Files
import sys
import csv
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup 
from selenium import webdriver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
## Selnium Custom Settings for colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(chrome_options=options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
## Year and month to scrape
year = input("Input the year you want to scrape from, eg. 2015: ")
month =input("Input the month you want to scrape from, eg. 2: ")
particularDay = int(input("Input the day you want to scrape from, eg. 1: "))
particularEnd = int(input("Input the day you want to scrape till, eg. 10: "))
articlePerDay = int(input("Enter the number of articles per day (Optimal 10):"))

## Opening the link
url = "https://timesofindia.indiatimes.com/archive/year-{},month-{}.cms".format(year,month)
driver.get(url)

## Generating the list of links of day
dayList = []
for x in range(2,7):
    for y in range(1,8):
        try:
            day = driver.find_element_by_xpath("/html/body/center/div[1]/div[3]/table[2]/tbody/tr[2]/td[1]/div[3]/table/tbody/tr[1]/td/span/div/table/tbody/tr[{}]/td[{}]/a".format(x,y))
            dayList.append(day.get_attribute('href'))
        except:
            continue
## Opening a csv file, scraping and storing the values.
for d in range(particularDay,particularEnd+1):
  fileName = 'TOIArticleArchive{}.csv'.format(d)
  with open(fileName, 'w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["Date","Author","Vertical","Headline","Description"])
      for day in dayList:
          if (dayList.index(day) != d-1):
            print("Skipping day", dayList.index(day)+1)
            continue
          ## iterate in dayList
          print("\n Running on day",day)
          articleLinks = []
          count = 1
          
          r = requests.get(day) 
          
          soup = BeautifulSoup(r.content, 'html5lib') 
          for a in soup.find_all('tbody')[2].find_all('a',href=True):
              if a.text: 
                  articleLinks.append(a['href'])
          print("Articles Found",len(articleLinks))
          if(len(articleLinks)== 0):
              print("No Articles Here")
              continue
          ## iterate in article Links
          count = 1
          for link in range(0,len(articleLinks)):
            if count>articlePerDay:
                break
            try:
              print("Scraping article {} of {}".format(count, articlePerDay))
              # Opening an article
              try:
                r = requests.get(articleLinks[link]) 
              except:
                print("Website error, ",articleLinks[link])
                continue
              soup = BeautifulSoup(r.content, 'html5lib') 
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              try:
                day = soup.find('div',attrs = {'class':'as_byline'}).find('div',attrs = {'data-plugin':"dateformat"}).text[9:]
              except:
                dayStore = soup.find('div',attrs = {'class':'_3Mkg- byline'}).text
                day = dayStore[dayStore.rfind("|")+1:]
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              try:
                author = soup.find('a',attrs = {'class':'auth_detail'}).text ## code for original
              except:
                try:
                  author = soup.find('div',attrs = {'class':'as_byline'}).find('div').text ## code for original
                except:
                  try:
                    author = soup.find('div',attrs = {'class':'_3Mkg- byline'}).find('span').text
                  except:
                    dayStore = soup.find('div',attrs = {'class':'_3Mkg- byline'}).text
                    author = dayStore[:dayStore.find("|")]
          
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              vertical = (" ".join(articleLinks[link].split("/")[5:len(articleLinks[0].split("/"))-3]))
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              headline = soup.find('title').text
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              try:
                article = soup.find("arttextxml").text
              except:
                try:
                  article = soup.find("Normal").text
                except:
                  try:
                    article = soup.find('div',attrs = {'class':'ga-headlines'}).text
                  except:
                    article = soup.find('div',attrs = {'class':'Normal'}).text
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              writer.writerow([day,author,vertical,headline,article])
              count+=1
            except:
              print("Skipping ",articleLinks[link],"Unknown Format")
              # pass
          break